# Transportation Hub Model Play

This file is a space for tinkering with prototype models using transportation hub data.

In [3]:
# Import libraries.
import numpy as np
import snowflake.connector
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [4]:
# Set global variables.
CREDENTIALS_FILE = 'credentials.txt'
DATABASE_ROOT = 'TRANSPORTATION_HUB.HUB'

In [5]:
# Read in Snowflake credentials.
with open(CREDENTIALS_FILE, 'r') as f:
    creds = f.readlines()
    USER = creds[0].strip()
    PASSWORD = creds[1].strip()
    ACCOUNT = creds[2].strip()

In [6]:
# Open a connection and test that it works.
con = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
)
cs = con.cursor()
cs.execute("SELECT current_version()")
one_row = cs.fetchone()
print(one_row[0])

8.21.3


### "Dumb" Delay Predictions via Linear Regression

**Description**: 

This is an intentionally bad model that attempts to predict a trip delay using a single-variable Linear Regression model.

**Notes**:
* When using [Time Series](https://www.tableau.com/learn/articles/time-series-analysis) data, you should generally use Time Series Analysis models like [Exponential Smoothing](https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html).

In [9]:
# Import data as a Pandas Dataframe and preview.
cs.execute(f"SELECT * FROM {DATABASE_ROOT}.TRIP_DELAYS")
trip_delay_data = cs.fetch_pandas_all()
trip_delay_data

,TRIP_DELAY_ID,YEAR,MONTH,DAY,DAY_OF_WEEK,TIME,TIMESTAMP,TRIP_ID,TRIP_STOP_SEQUENCE,STOP_LOCATION_ID,DELAY
0,179957,2023,9,19,Tue,18:30:23,2023-09-19 18:30:23,12769108,30,8344,253.0
1,179974,2023,9,19,Tue,18:36:16,2023-09-19 18:36:16,12769108,32,8346,233.0
2,179975,2023,9,19,Tue,18:36:33,2023-09-19 18:36:33,12769108,32,8346,231.0
3,179986,2023,9,19,Tue,18:40:07,2023-09-19 18:40:07,12769108,33,8347,236.0
4,180000,2023,9,19,Tue,18:45:29,2023-09-19 18:45:29,12769108,35,8349,216.0
...,...,...,...,...,...,...,...,...,...,...,...
1442288,1442269,2023,9,25,Mon,07:26:39,2023-09-25 07:26:39,12810486,14,13716,310.0
1442289,1442275,2023,9,25,Mon,07:28:57,2023-09-25 07:28:57,12810486,15,13717,309.0
1442290,1442277,2023,9,25,Mon,07:29:27,2023-09-25 07:29:27,12810486,15,13717,283.0
1442291,1442288,2023,9,25,Mon,07:33:31,2023-09-25 07:33:31,12810486,16,13718,299.0


In [10]:
# Import data as a Pandas Dataframe and preview.
cs.execute(f"SELECT * FROM {DATABASE_ROOT}.TRIPS")
trip_data = cs.fetch_pandas_all()
trip_data

,TRIP_ID,ROUTE_ID,ROUTE_DIRECTION
0,12768209,94,Outbound
1,12764244,74,Inbound
2,12757617,34,Outbound
3,12762205,70,Inbound
4,12762383,71,Outbound
...,...,...,...
19582,12790977,32,Outbound
19583,12800772,87,Outbound
19584,12774812,290,Outbound
19585,12752025,12,Inbound


In [11]:
trip_delay_data_merged = trip_delay_data.merge(trip_data, on='TRIP_ID')
trip_delay_data_merged 

,TRIP_DELAY_ID,YEAR,MONTH,DAY,DAY_OF_WEEK,TIME,TIMESTAMP,TRIP_ID,TRIP_STOP_SEQUENCE,STOP_LOCATION_ID,DELAY,ROUTE_ID,ROUTE_DIRECTION
0,179957,2023,9,19,Tue,18:30:23,2023-09-19 18:30:23,12769108,30,8344,253.0,100,Outbound
1,179974,2023,9,19,Tue,18:36:16,2023-09-19 18:36:16,12769108,32,8346,233.0,100,Outbound
2,179975,2023,9,19,Tue,18:36:33,2023-09-19 18:36:33,12769108,32,8346,231.0,100,Outbound
3,179986,2023,9,19,Tue,18:40:07,2023-09-19 18:40:07,12769108,33,8347,236.0,100,Outbound
4,180000,2023,9,19,Tue,18:45:29,2023-09-19 18:45:29,12769108,35,8349,216.0,100,Outbound
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308678,1442269,2023,9,25,Mon,07:26:39,2023-09-25 07:26:39,12810486,14,13716,310.0,290,Inbound
1308679,1442275,2023,9,25,Mon,07:28:57,2023-09-25 07:28:57,12810486,15,13717,309.0,290,Inbound
1308680,1442277,2023,9,25,Mon,07:29:27,2023-09-25 07:29:27,12810486,15,13717,283.0,290,Inbound
1308681,1442288,2023,9,25,Mon,07:33:31,2023-09-25 07:33:31,12810486,16,13718,299.0,290,Inbound


In [27]:
import datetime
trip_delay_data_merged['TIMESTAMP_UNIX'] = trip_delay_data_merged.TIMESTAMP.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
trip_delay_data_merged

,TRIP_DELAY_ID,YEAR,MONTH,DAY,DAY_OF_WEEK,TIME,TIMESTAMP,TRIP_ID,TRIP_STOP_SEQUENCE,STOP_LOCATION_ID,DELAY,ROUTE_ID,ROUTE_DIRECTION,TIMESTAMP_UNIX
0,179957,2023,9,19,Tue,18:30:23,2023-09-19 18:30:23,12769108,30,8344,253.0,100,Outbound,1.695148e+09
1,179974,2023,9,19,Tue,18:36:16,2023-09-19 18:36:16,12769108,32,8346,233.0,100,Outbound,1.695149e+09
2,179975,2023,9,19,Tue,18:36:33,2023-09-19 18:36:33,12769108,32,8346,231.0,100,Outbound,1.695149e+09
3,179986,2023,9,19,Tue,18:40:07,2023-09-19 18:40:07,12769108,33,8347,236.0,100,Outbound,1.695149e+09
4,180000,2023,9,19,Tue,18:45:29,2023-09-19 18:45:29,12769108,35,8349,216.0,100,Outbound,1.695149e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308678,1442269,2023,9,25,Mon,07:26:39,2023-09-25 07:26:39,12810486,14,13716,310.0,290,Inbound,1.695627e+09
1308679,1442275,2023,9,25,Mon,07:28:57,2023-09-25 07:28:57,12810486,15,13717,309.0,290,Inbound,1.695627e+09
1308680,1442277,2023,9,25,Mon,07:29:27,2023-09-25 07:29:27,12810486,15,13717,283.0,290,Inbound,1.695627e+09
1308681,1442288,2023,9,25,Mon,07:33:31,2023-09-25 07:33:31,12810486,16,13718,299.0,290,Inbound,1.695627e+09


In [35]:
# Filter out records with NaN inputs or outputs.
filtered_trip_delay_data = trip_delay_data_merged.dropna(subset=['TIMESTAMP_UNIX', 'DELAY', 'ROUTE_ID'])
print(f"{len(trip_delay_data_merged['TIMESTAMP_UNIX']) - len(filtered_trip_delay_data['DELAY'])} records removed.")
filtered_trip_delay_data.dtypes

1836 records removed.


TRIP_DELAY_ID                  int32
YEAR                           int16
MONTH                           int8
DAY                             int8
DAY_OF_WEEK                   object
TIME                          object
TIMESTAMP             datetime64[ns]
TRIP_ID                        int32
TRIP_STOP_SEQUENCE             int16
STOP_LOCATION_ID               int16
DELAY                        float64
ROUTE_ID                       int16
ROUTE_DIRECTION               object
TIMESTAMP_UNIX               float64
dtype: object

In [34]:
# Subset features (inputs) and response (outputs).
features = filtered_trip_delay_data[['TIMESTAMP_UNIX', 'ROUTE_ID']].to_numpy()
response = filtered_trip_delay_data['DELAY'].to_numpy()
features

array([[1.69514822e+09, 1.00000000e+02],
       [1.69514858e+09, 1.00000000e+02],
       [1.69514859e+09, 1.00000000e+02],
       ...,
       [1.69562697e+09, 2.90000000e+02],
       [1.69562721e+09, 2.90000000e+02],
       [1.69561770e+09, 2.90000000e+02]])

In [30]:
# # Transform features into a numeric value for easier use.
# def datetime_to_unix(input):
#     return input.astype('datetime64[s]').astype('int')

# transformed_features = np.apply_along_axis(datetime_to_unix, 0, features['TIMESTAMP'])
transformed_features = features

In [33]:
# Split the data into training and testing sets.
features_train, features_test, response_train, response_test = train_test_split(transformed_features, response, test_size=0.2, random_state=777)

# Feed training data into the model.
model = LinearRegression().fit(features_train.reshape(-1, 2), response_train)

# Spit out predictions.
preds = model.predict(features_test.reshape(-1, 2))

# Evaluate the model. Remember, a low MSE is good, and an R2 of 1.0 is good.
print("Mean squared error: %.2f" % mean_squared_error(response_test, preds))
print("Coefficient of determination: %.2f" % r2_score(response_test, preds))


Mean squared error: 53914.68
Coefficient of determination: 0.00
